In [1]:
import sys 

In [ ]:
cd '/srv/share3/hagrawal9/project/habitat/habitat-api/'

In [ ]:
import gzip
import json
import os
import sys
from typing import Any, Dict, List, Optional, Type
from copy import deepcopy

import attr
import cv2
import git
import magnum as mn
import numpy as np

# %matplotlib inline
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image


import habitat
import habitat_sim
from habitat.config import Config
from habitat.core.registry import registry
from habitat_sim.utils import viz_utils as vut
from habitat.utils.visualizations import maps, fog_of_war
from habitat.utils.visualizations.utils import observations_to_image

from rearrangement.utils.planner import (
    compute_traversable_map,
    compute_distance_using_fmm,
    find_dist_from_map,
    compute_distance_mat_using_navmesh,
    compute_distance_mat_using_fmm,
    find_shortest_path_for_multiple_objects
)
from rearrangement.utils.visualization import (
    get_top_down_map
)
from rearrangement.utils.geometry import (
     geodesic_distance
)
from collections import defaultdict

In [4]:
from habitat.datasets.rearrangement.rearrangement_dataset import RearrangementDatasetV0
from habitat.tasks.rearrangement.rearrangement_task import RearrangementEpisode

In [41]:
repo = git.Repo(".", search_parent_directories=True)
dir_path = repo.working_tree_dir
# %cd $dir_path
data_path = os.path.join(dir_path, "data")
output_directory = "data/tutorials/output/"  # @param {type:"string"}
output_path = os.path.join(dir_path, output_directory)

In [42]:
config = habitat.get_config("configs/tasks/rearrangement_gibson.yaml")

In [43]:
config.defrost()
config.TASK.MEASUREMENTS = ['OBJECT_TO_GOAL_DISTANCE', 'AGENT_TO_OBJECT_DISTANCE', 'TOP_DOWN_MAP']
config.TASk.
config.freeze()

In [44]:
config.TASK.TOP_DOWN_MAP.MAP_RESOLUTION

256

In [45]:
try:
    env.close()
except:
    pass
env = habitat.Env(config)
fog_of_war_mask = None
pickup_order = {}


2020-10-18 18:29:57,271 Initializing dataset RearrangementDataset-v0
2020-10-18 18:30:01,017 initializing sim RearrangementSim-v0
2020-10-18 18:30:05,233 Initializing task RearrangementTask-v0


In [46]:
# # Scratch Code
# for objid in env._sim.get_existing_object_ids():
#     if objid != env._sim.agent_object_id:
#         episode_obj_id = env._task.sim_object_to_objid_mapping[objid]
#         dist = geodesic_distance(env._sim.pathfinder, agent_pos, env.current_episode.objects[episode_obj_id].position)
#         if dist == np.inf:
#             print("Voila!!")


In [47]:
def compute_oracle_pickup_order(env):
    obs = env.reset()
    metrics = env.get_metrics()
    episode = env.current_episode
    metrics = env.get_metrics()

    agent_pos = env._sim.get_agent(0).get_state().position
    object_positions = [obj.position for obj in episode.objects]
    goal_positions = [obj.position for obj in episode.goals]
    
#     dist_mat = compute_distance_mat_using_navmesh(env._task._simple_pathfinder, agent_pos, object_positions, goal_positions)
#     route_indexes_navmesh, pickup_order_navmesh = find_shortest_path_for_multiple_objects(dist_mat)
    
    top_down_map, fog_of_war_mask = get_top_down_map(
        env, env._task._simple_pathfinder, ignore_objects=True, fog_of_war_mask=None, draw_fow=False, 
        draw_agent=False, draw_object_start_pos=False, draw_object_final_pos=False, draw_object_curr_pos=False
    )
    
    
    a_y, a_x = maps.to_grid(
        agent_pos[2],
        agent_pos[0],
        top_down_map.shape[0:2],
        sim=env._sim,
    )
    grid_object_positions = []
    grid_goal_positions = []

    for i, obj_pos in enumerate(object_positions):
        tdm_pos = maps.to_grid(
            obj_pos[2],
            obj_pos[0],
            top_down_map.shape[0:2],
            sim=env._sim,
        )
        grid_object_positions.append(tdm_pos)

    for i, goal_pos in enumerate(goal_positions):
        tdm_pos = maps.to_grid(
            goal_pos[2],
            goal_pos[0],
            top_down_map.shape[0:2],
            sim=env._sim,
        )

        grid_goal_positions.append(tdm_pos)

    tdmap = np.copy(top_down_map[:, :, 0])
    tdmap = tdmap / np.max(tdmap)
    dist_mat_map = compute_distance_mat_using_fmm(tdmap, [a_y, a_x], grid_object_positions, grid_goal_positions)
    
    
    route_indexes_map, pickup_order_map = find_shortest_path_for_multiple_objects(dist_mat_map/3)
    
    return {
        'episode_id': episode.episode_id, 
        'scene_id': episode.scene_id,
#         'pickup_order': pickup_order_navmesh,
        'pickup_order_fmm': pickup_order_map 
    }

In [ ]:
while(len(pickup_order)!=35000):
    data = compute_oracle_pickup_order(env)
    pickup_order[data['episode_id'] + '_' + data['scene_id']] = data
    print('\rEpisode ID: {} Done: {} '.format(data['episode_id'], len(pickup_order)), end=" ")

Episode ID: 375 Done: 2518  

In [ ]:
with open('data/pickup_order_tdmap_train.json','w') as f:
    json.dump(pickup_order, f)

In [48]:
with open('data/pickup_order_tdmap.json','r') as f:
    pickup_order = json.load(f)

In [49]:
updated_scene_data = []
split = 'train'

for filename in os.listdir('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split)):
    scene_id = filename.split('.')[0].split('_')[-1]
    with gzip.open('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split) + filename, "rt") as f:
        scene_data = json.load(f)
    updated_scene_data = deepcopy(scene_data)
    
    for i, episode in enumerate(scene_data['episodes']):
        episode_id = episode['episode_id']
        assert updated_scene_data['episodes'][i]['episode_id'] == episode['episode_id']
        assert updated_scene_data['episodes'][i]['scene_id'] == episode['scene_id']
        updated_scene_data['episodes'][i]['pickup_order'] = pickup_order[str(episode['episode_id']) + "_" + episode['scene_id']]['pickup_order']
    
    with gzip.open('data/datasets/rearrangement/gibson/v1/{}/content/'.format(split) + filename, "wt") as f:
        json.dump(updated_scene_data, f)
        
    print(filename)

rearrangement_v1_train_n=1000_o=5_Monson.json.gz
rearrangement_v1_train_n=1000_o=5_Scandinavia.json.gz
rearrangement_v1_train_n=1000_o=5_Hobson.json.gz
rearrangement_v1_train_n=1000_o=5_Barboursville.json.gz
rearrangement_v1_train_n=1000_o=5_Mentasta.json.gz
rearrangement_v1_train_n=1000_o=5_Avonia.json.gz
rearrangement_v1_train_n=1000_o=5_Ewell.json.gz
rearrangement_v1_train_n=1000_o=5_Gilbert.json.gz
rearrangement_v1_train_n=1000_o=5_Sawpit.json.gz
rearrangement_v1_train_n=1000_o=5_Montreal.json.gz
rearrangement_v1_train_n=1000_o=5_Orason.json.gz
rearrangement_v1_train_n=1000_o=5_Hainesburg.json.gz
rearrangement_v1_train_n=1000_o=5_Helix.json.gz
rearrangement_v1_train_n=1000_o=5_Westerville.json.gz
rearrangement_v1_train_n=1000_o=5_Arona.json.gz
rearrangement_v1_train_n=1000_o=5_Delton.json.gz
rearrangement_v1_train_n=1000_o=5_Roeville.json.gz
rearrangement_v1_train_n=1000_o=5_Euharlee.json.gz
rearrangement_v1_train_n=1000_o=5_Hominy.json.gz
rearrangement_v1_train_n=1000_o=5_Bowlus.j